In [ ]:
import numpy as np
import pandas as pd
import io
import plotly.express as px
import plotly.offline as pyo

In [ ]:
df = pd.read_csv("2 Machine Learning Dataset\\2 DataScientistJobs.csv")

# 看Data Information ，把info函數轉成我們好看的dataframe

In [ ]:
def get_data_description(data: pd.DataFrame):
    buffer = io.StringIO()
    data.info(buf=buffer)
    lines = buffer.getvalue().splitlines()
    details = (pd.DataFrame([x.split() for x in lines[5:-2]], columns=lines[3].split())
               .drop('Count', axis=1)
               .rename(columns={'Non-Null': 'Non-Null Count'})
               .rename(columns={'#': 'number'}))
    unique_value = []
    for i in data.columns:
        unique_value.append(len(data[i].unique()))
    details['unique'] = unique_value
    return details

因為在上面的函數中，我使用了split()，沒有特別調，但是在這次檔案中columns中是有空白的，因此會出錯，我將columns有空白的部分加了底線， 讓我能使用上面的函數。

In [ ]:
y = [x.replace(" ", "_") for x in df.columns]
df.columns = y
df_final = df.copy()
df_final = df_final.iloc[:, 2:]  #去除index和沒用的column
df_final

In [ ]:
df_final.describe()

In [ ]:
df_detail = get_data_description(df)
df_detail

| 變數名稱              | 變數意義   |     |
|-------------------|--------|-----|
| Job_Title         | 工作名稱   | v   |
| Salary_Estimate   | 薪資間距   | v   |
| Job_Description   | 工作敘述   | x   |
| Rating            | 評分     | x   |
| Company_Name      | 公司名稱   | v   |
| Location          | 工作地點   | v   |
| Headquarters      | 公司總部地點 | v   |
| Size              | 公司人數大小 | v   |
| Founded           | 資金     |     |
| Type_of_ownership | 公司法人狀況 |     |
| Industry          | 何種產業   | x   |
| Revenue           | 公司獲利   | v   |
| Sector            | 部門     | x   |
| Competitors       | 競爭者    | x   |
| Easy_Apply        || v      |

In [ ]:
df.Type_of_ownership.unique()

In [ ]:
df.Competitors.unique()

# Simple Feature Engineering
要做EDA之前，要先針對比較重要的變數做特別的處理。

## 薪資間距處理(Salary Estimate)

Salary Estimate要特別去處理一種是，Per Hour的薪資 因此我把它統一弄成年薪的樣子，並假設一年是52week，一周工作40hours

In [ ]:
def salary_format(st):
    """
    :param st:Salary裡的字串
    :type st:
    :return: 處理後的字串
    :rtype: str
    """
    hours_week = 40
    weeks_year = 52
    if " Per Hour" in st:
        st_final = st.replace("(Glassdoor est.)", "").replace("(Employer est.)", "")
        st_final = st_final.replace(" Per Hour", "")
        st_up = int(st_final.split("-")[0].replace('$', '')) * hours_week * weeks_year
        st_down = int(st_final.split("-")[1].replace('$', '')) * hours_week * weeks_year

    else:
        st_final = st.replace("(Glassdoor est.)", "").replace("(Employer est.)", "")
        st_up = int(
            st_final.split("-")[0].replace("$", "").replace("K", "000")
        )
        st_down = int(
            st_final.split("-")[1].replace("$", "").replace("K", "000")
        )
    return [st_up, st_down]


df_final["Salary_cond"] = df_final['Salary_Estimate'].apply(lambda st: salary_format(st))
df_final[['Salary_cond', 'Salary_Estimate']]

In [ ]:
df.Company_Name.unique()

## Company_name
由上面可以發現，Company_name此欄位不只有公司名稱，還有公司的評分，需特別取出

In [ ]:
def get_companyrating(strs):
    """
    取得公司評分
    :param strs:
    :type strs:
    :return:
    :rtype:
    """
    if "\n" in strs:
        strs = strs.split('\n')[1]
    else:
        strs = 'No Info'
    return strs


def get_company_name(strs):
    """
    取得公司名稱
    :param strs:
    :type strs:
    :return:
    :rtype:
    """
    if "\n" in strs:
        strs = strs.split('\n')[0]
    else:
        strs = strs
    return strs


df_final['Company_Rating'] = df_final['Company_Name'].apply(lambda strs: get_companyrating(strs))
df_final['C_Name'] = df_final['Company_Name'].apply(lambda strs: get_company_name(strs))


In [ ]:
df_final[['Company_Name', 'Company_Rating', 'C_Name']]

## Size
從下面我們可以看出要如何分出公司人數的上下間距。

In [ ]:
df.Size.unique()

In [ ]:
def size_of_company(strs):
    """
    將Size中的字串轉置成公司人數的上下限numeric資料
    :param strs:Size中的字串
    :type strs:str
    :return:list of numeric
    :rtype:list
    """
    if 'to' in strs:
        top = int(
            strs.split('to')[1].split(" ")[1].strip(' ')
        )
        down = int(
            strs.split('to')[0].strip(" ")
        )
    elif '+' in strs:
        down = int(
            strs.split('+')[0].strip("")
        )
        top = np.inf
    else:
        top = np.nan
        down = np.nan
    return [down, top]


df_final["Size_top"] = df_final['Size'].apply(lambda x: size_of_company(x)[1])
df_final["Size_down"] = df_final['Size'].apply(lambda x: size_of_company(x)[0])


In [ ]:
test = "501 to 1000 employees"
test.split("to")[1].split(" ")[1]

## Easy_apply

可以看出此欄位只有兩種職，一種就是-1一種就是True，我們可以將其轉換成bool

In [ ]:
df.Easy_Apply.unique()

In [ ]:
def format_easyapply(strs):
    if strs == "-1":
        strs = False
    else:
        strs = True
    return strs


In [ ]:
df_final['Easy_Apply_f'] = df['Easy_Apply'].apply(lambda x: format_easyapply(x))

## City and State

因為此資料集是探究美國的data scientist jobs的薪資
因此公司總部在甚麼城市，哪一個州也是一個可以提出的特徵

同理工作地點也是，
而我們還能由此在延伸，工作地點和總部不同，或許可能算是外派，對薪資的影響是否顯著等。

In [ ]:
df.Location.unique()

In [ ]:
df.Headquarters.unique()

In [ ]:
def get_location(strs: str):
    [city, state] = [strs.split(',')[0], strs.split(',')[1].strip('')]
    return [city, state]


def get_headquarter(strs: str):
    if strs != '-1':
        [city, state] = [strs.split(',')[0], strs.split(',')[1].strip('')]
    else:
        [city, state] = ['No Info', 'No info']
    return [city, state]

In [ ]:
df_final['Location_City'] = df_final['Location'].apply(lambda city: get_location(city)[0])
df_final['Location_State'] = df_final['Location'].apply(lambda state: get_location(state)[0])
df_final['Headquarters_City'] = df_final['Headquarters'].apply(lambda city: get_headquarter(city)[0])
df_final['Headquarters_State'] = df_final['Headquarters'].apply(lambda state: get_headquarter(state)[0])

## Job Title

觀察之後發現，此欄位的前面為工作名稱，而後面通常為其工作的細節，用','去分隔。

In [ ]:
df.Job_Title.unique()

In [ ]:
def format_job(strs: str):
    """
    將Job_title裡面的字串做處理，返回一個list
    日後若想再對其做進一步處理，寫成function會比較方便。
    :param strs: Job_title裡面的string
    :type strs: str
    :return: list of string
    :rtype: list
    """
    strs = strs.split(',')
    strs=strs[0]
    if len(strs)>25:
        strs=strs[0:25]

    return strs

In [275]:
df_final['Job_Title_real'] = df_final['Job_Title'].apply(lambda title: format_job(title))

In [276]:
df_final['Job_Title_real'].value_counts(ascending=False)

Data Scientist               335
Data Engineer                268
Data Analyst                 264
Senior Data Scientist        104
Machine Learning Engineer     72
                            ... 
Gilbert Data Science Tuto      1
Chandler Data Science Tut      1
Cloud Data Engineer - Sol      1
Cloud Data Engineer - Sen      1
Patient Safety Physician       1
Name: Job_Title_real, Length: 1609, dtype: int64

## Revenue
和Salary estimate類似的處理方法。

In [277]:
df.Revenue.unique()

array(['Unknown / Non-Applicable', '$100 to $500 million (USD)',
       '$500 million to $1 billion (USD)', '$10+ billion (USD)',
       '$25 to $50 million (USD)', '$1 to $5 million (USD)',
       '$5 to $10 billion (USD)', '-1', '$5 to $10 million (USD)',
       '$50 to $100 million (USD)', '$1 to $2 billion (USD)',
       '$2 to $5 billion (USD)', 'Less than $1 million (USD)',
       '$10 to $25 million (USD)'], dtype=object)

In [278]:
he='$500 million to $1 billion (USD)'

he.replace('(USD)', '').replace('$', '').split("to")[1].split(" ")

['', '1', 'billion', '']

In [279]:
he2="$5 to $10 million (USD)"
he2.replace('(USD)', '').replace('$', '').split(" ")

['5', 'to', '10', 'million', '']

In [280]:
def revenue_format(strs: str):
    strs = strs.replace('(USD)', '').replace('$', '')
    bil = 1000000000
    mil = 1000000
    if '+' not in strs:
        if 'billion' in strs and 'million' in strs:
            down=int(
                strs.split("to")[0].split(' ')[0]
            )*mil
            top=int(
                strs.split("to")[1].split(' ')[1]
            )*bil
        elif 'billion' in strs:
            down = int(
                strs.split(" ")[0]
            ) * bil
            top = int(
                strs.split(" ")[2]
            ) * bil
        elif 'million' in strs:
            if 'Less' in strs:
                top=int(
                    strs.split(" ")[2]
                )*mil
                down=0
            else:
                down = int(
                    strs.split(" ")[0]
                ) * mil
                top = int(
                    strs.split(" ")[2]
                ) * mil

        else:
            [down, top] = [0, 0]
    else:
        if 'billion' in strs:
            down = int(
                strs.split("+")[0]
            )*bil
            top=np.inf
        elif 'million' in strs:
            down =int(
                strs.split("+")[0]
            )
            top=np.inf
        else:
            [top, down] = [0, 0]
    return [down,top]





In [281]:
df_final['Revenue_top']=df_final['Revenue'].apply(lambda revenue:revenue_format(revenue)[1])
df_final['Revenue_down']=df_final['Revenue'].apply(lambda revenue:revenue_format(revenue)[0])



In [282]:
df_final[['Revenue','Revenue_down','Revenue_top',]]

,Revenue,Revenue_down,Revenue_top
0,Unknown / Non-Applicable,0,0.000000e+00
1,Unknown / Non-Applicable,0,0.000000e+00
2,Unknown / Non-Applicable,0,0.000000e+00
3,Unknown / Non-Applicable,0,0.000000e+00
4,Unknown / Non-Applicable,0,0.000000e+00
...,...,...,...
3904,$10 to $25 million (USD),10000000,2.500000e+07
3905,$50 to $100 million (USD),50000000,1.000000e+08
3906,$100 to $500 million (USD),100000000,5.000000e+08
3907,Less than $1 million (USD),0,1.000000e+06


## Industry

不需特別處理

In [283]:
df_final.Industry.unique()

array(['Travel Agencies', 'Health, Beauty, & Fitness', '-1', 'Internet',
       'Advertising & Marketing', 'Insurance Carriers',
       'Investment Banking & Asset Management',
       'Enterprise Software & Network Solutions', 'Other Retail Stores',
       'Consulting', 'Research & Development',
       'Motion Picture Production & Distribution', 'Brokerage Services',
       'IT Services', 'Computer Hardware & Software', 'Security Services',
       'Casual Restaurants', 'Food & Beverage Stores', 'Food Production',
       'Health Fundraising Organizations',
       'Home Furniture & Housewares Stores',
       'Financial Analytics & Research', 'Education Training Services',
       'Lending', 'Social Assistance', 'Publishing', 'Federal Agencies',
       'Health Care Services & Hospitals',
       'Oil & Gas Exploration & Production', 'Staffing & Outsourcing',
       'Venture Capital & Private Equity',
       'Building & Personnel Services', 'Video Games', 'Accounting',
       'Consumer Produ

## Sector
不需特別處理

In [284]:
df_final.Sector.unique()

array(['Travel & Tourism', 'Consumer Services', '-1',
       'Information Technology', 'Business Services', 'Insurance',
       'Finance', 'Retail', 'Media', 'Restaurants, Bars & Food Services',
       'Agriculture & Forestry', 'Non-Profit', 'Education', 'Government',
       'Health Care', 'Oil, Gas, Energy & Utilities',
       'Accounting & Legal', 'Manufacturing', 'Real Estate',
       'Biotech & Pharmaceuticals', 'Arts, Entertainment & Recreation',
       'Aerospace & Defense', 'Construction, Repair & Maintenance',
       'Transportation & Logistics', 'Telecommunications',
       'Mining & Metals'], dtype=object)

## Type_of_ownership

In [285]:
df_final.Type_of_ownership.unique()

array(['Company - Private', 'Unknown', 'Company - Public',
       'Nonprofit Organization', '-1', 'Government', 'Hospital',
       'Self-employed', 'Subsidiary or Business Segment', 'Contract',
       'School / School District', 'Private Practice / Firm',
       'College / University', 'Franchise', 'Other Organization'],
      dtype=object)

# 統計圖表分析

In [286]:
Top_jobs_counts=df_final.C_Name.value_counts().sort_values(ascending=False).head(20).reset_index()
Top_jobs_counts

,index,C_Name
0,Apple,60
1,IBM,57
2,Amazon,47
3,"Staffigo Technical Services, LLC",28
4,Facebook,25
5,Diverse Lynx,25
6,Solekai Systems Corp,24
7,Southwest Research Institute,23
8,Varsity Tutors,21
9,Humana,20


In [287]:
fig = px.bar(Top_jobs_counts,x='index',y='C_Name',
             labels={'C_Name':'Counts',
                     'index':'Company Name',                                })

fig.show()

In [288]:
top_title=df_final.Job_Title_real.value_counts().sort_values(ascending=False).head(20).reset_index()
top_title

,index,Job_Title_real
0,Data Scientist,335
1,Data Engineer,268
2,Data Analyst,264
3,Senior Data Scientist,104
4,Machine Learning Engineer,72
5,Business Intelligence Ana,62
6,Senior Data Analyst,52
7,Big Data Engineer,42
8,Scientist,40
9,Senior Data Engineer,37


In [292]:
fig = px.bar(top_title, x='index', y='Job_Title_real',
             labels={'C_Name': 'Counts',
                     'index': 'Job Name', })

fig.show()